# **Гипотезы**

In [18]:
import pandas as pd
import numpy as np
from scipy import stats
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

Загрузим даннные.

In [19]:
movies = pd.read_csv('movies_dataset.csv')
visuals = pd.read_csv('visuals_dataset.csv')

В ходе EDA мы обнаружили ряд закономерностей, которые бы хотелось проверить на статистическую значимость и вообще на присутствие в данных.

Наши гипотезы:
1. Аудитория активнее взаимодействует с более свежими фильмами по сравнению со старыми фильмами
2. Титулованность фильма не будет влиять на отклик аудитории или будет влиять отрицательно
3. Если фильм выиграл Оскар, то скорее всего у него есть и еще одна награда из двух оставшихся: Золотой глобус или Золотая пальмовая ветвь
4. Люди больше взаимодействуют с постами, включающими только картинки, нежели чем с постами, посвященными фильмам

Приступим к проверке.

##### Гипотеза 1

**Аудитория активнее взаимодействует с более свежими фильмами по сравнению со старыми фильмами**

Перменными, отражающими взаимодействие, будут log_likes, log_reposts, log_comments и ER.

\begin{array}{l}
\text{Нулевая гипотеза } (H_0): \mu_0 = \mu_1 \\
\text{Альтернативная гипотеза } (H_1): \mu_0 > \mu_1
\end{array}

Так как разрыв в годах выпуска между самым старым и новым фильмами 121 год, то будем считат фильм новым, если он был выпущен в 1991 году или позднее. Разделим данные на две выборки: со старыми и новыми фильмами.

In [20]:
movies_old = movies[movies['year'] < 1991]
movies_new = movies[movies['year'] >= 1991]

Проверим, равны ли у выделенных выборок дисперсии.

In [21]:
np.var(movies_old, ddof=1)

likes            1.026698e+04
reposts          9.009269e+03
views            7.251084e+07
comments         2.489698e+01
attachments      3.760200e-01
year             1.277992e+02
oscar_winner     7.604340e-03
oscar_nominee    4.733702e-02
palme_winner     7.604340e-03
globe_winner     1.509169e-02
movie_flg        0.000000e+00
log_likes        4.605859e-01
log_reposts      7.787391e-01
log_views        2.449293e-01
log_comments     7.891722e-01
reactions        3.589965e+04
ER               3.674786e-01
prize            3.737826e-02
other_award      2.246205e-02
dtype: float64

In [22]:
np.var(movies_new, ddof=1)

likes            1.871790e+04
reposts          9.509907e+03
views            8.503812e+07
comments         4.323125e+01
attachments      3.342910e-01
year             9.608565e+01
oscar_winner     1.343887e-02
oscar_nominee    7.048928e-02
palme_winner     2.132449e-02
globe_winner     1.343887e-02
movie_flg        0.000000e+00
log_likes        5.723889e-01
log_reposts      6.781536e-01
log_views        2.372845e-01
log_comments     9.255400e-01
reactions        4.939633e+04
ER               4.568789e-01
prize            6.299609e-02
other_award      3.417101e-02
dtype: float64

Дисперсии не равны. Будем это учитывать при проведении теста.
Для проверки гипотезы будем использовать двувыборочный t-тест, так как данные являются количественными, распределены нормально, а выборки являются независимыми. Будем проверять гипотезу на уровне значимости 5%.

In [23]:
stats.ttest_ind(movies_new['log_likes'], movies_old['log_likes'], alternative = 'greater', equal_var = False)

Ttest_indResult(statistic=7.281100046033788, pvalue=5.261841944783864e-13)

In [24]:
stats.ttest_ind(movies_new['ER'], movies_old['ER'], alternative = 'greater', equal_var = False)

Ttest_indResult(statistic=6.9944275447686675, pvalue=3.590441583408903e-12)

In [25]:
stats.ttest_ind(movies_new['log_reposts'], movies_old['log_reposts'], alternative = 'greater', equal_var = False)

Ttest_indResult(statistic=6.569269295369289, pvalue=5.974830477310235e-11)

In [26]:
stats.ttest_ind(movies_new['log_comments'], movies_old['log_comments'], alternative = 'greater', equal_var = False)

Ttest_indResult(statistic=7.07762769354463, pvalue=2.097854379740498e-12)

**Вывод:** так как во всех случаях у нас получился мизерный p-value, то мы отвергаем нулевую гипотезу о равенстве среднего количества лайков/репостов/комментариев/ER для новых и старых фильмов в пользу альтернативной о том, что данные показатели больше для более свежих фильмов. Гипотеза **подтвердилась**.

##### Гипотеза 2

**Титулованность фильма не будет влиять на отклик аудитории или будет влиять отрицательно**

Здесь мы будем смотреть на переменные prize и ER. Уровень значимости снова 5%.

\begin{array}{l}
\text{Нулевая гипотеза } (H_0): \rho \leq 0 \\
\text{Альтернативная гипотеза } (H_1): \rho > 0
\end{array}

Сначала посмотрим наблюдаемый коэффициент корреляции.

In [30]:
 o_corr = movies['prize'].corr(movies['ER'])
 o_corr

0.014865169239513578

Теперь нам надо понять, является ли полученный результат статистически значимым. Для этого проведем тест перестановок.

In [29]:
results = []
y = movies['ER']
for i in range(10000):
    x = np.random.permutation(movies['prize'])
    corr_coef = np.corrcoef(x, y)[0, 1]
    results.append(corr_coef)

pvalue = np.sum(np.abs(results) >= np.abs(o_corr))/10000
pvalue 

0.7089

**Вывод:** так как мы получили p-value, который гораздо больше установленного уровня значимости (5%), то мы не отвергаем нулевую гипотезу о существовании корреляции, которая либо равна (очень близка к 0), либо меньше 0

##### Гипотеза 3

**Если фильм номинирован на Оскар, то скорее всего у него есть и еще одна награда из двух оставшихся**

Здесь обратим внимание на переменные oscar_winner и other award. Уровень значимости равен 5%.

\begin{array}{l}
\text{Нулевая гипотеза } (H_0): присуждение\ Оскара\ и\ обладание\ другими\ наградами\ -\ независимые\ признаки \\
\text{Альтернативная гипотеза } (H_1): присуждение\ Оскара\ и\ обладание\ другими\ наградами\ не\ являются\ независимыми\ признаками
\end{array}

Для проверки гипотезы будем использовать критерий независимости хи-квадрат, так как данные булевые и нам надо проверить наличие зависимости между признаками.

In [31]:
table = pd.crosstab(movies['oscar_winner'], movies['other_award'])
stats.chi2_contingency(table)

Chi2ContingencyResult(statistic=53.4243385362577, pvalue=2.687465684202394e-13, dof=1, expected_freq=array([[6.04211111e+02, 1.87888889e+01],
       [6.78888889e+00, 2.11111111e-01]]))

**Вывод:** у нас очень маленький p-value, поэтому мы отвергаем нулевую гипотезу о том, что нет зависимости между присуждением Оскара и получением "Золотого глобуса" или "Пальмовой ветви". Наша первоначальная гипотеза **подтвердилась**.

##### Гипотеза 4

**Люди больше взаимодействуют с постами, включающими только картинки, нежели чем с постами, посвященными фильмам**


\begin{array}{l}
\text{Нулевая гипотеза } (H_0): \mu_0 = \mu_1 \\
\text{Альтернативная гипотеза } (H_1): \mu_0 > \mu_1
\end{array}

Тут будем использовать переменные movie_flg и ER. Сначала посмотрим, равны ли дисперсии ER для постов с информацией о фильмах и просто картинками.

In [32]:
print(movies['ER'].var(ddof = 1))
print(visuals['ER'].var(ddof = 1))

0.4505296322022408
0.6215350137651525


Дисперсии не равны. Учтем это при проведении теста. Будем использовать t-тест. Уровень значимости снова 5%.

In [33]:
stats.ttest_ind(visuals['ER'], movies['ER'], alternative = 'greater', equal_var = False)

Ttest_indResult(statistic=2.9717187575918502, pvalue=0.0015460989077053977)

**Вывод:** p-value меньше установленных 5%, поэтому мы отвергаем нулевую гипотезу в сторону альтернативной и можем говорить о том, что люди больше взаимодействуют с постами, включающими только картинки. Гипотеза **подтвердилась**.